In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

!git clone https://github.com/Sitnich/ssyp-2022-materials

!unzip ssyp-2022-materials/data/pokemon.zip -d pokemon/

Mounted at /content/gdrive
Cloning into 'ssyp-2022-materials'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 34 (delta 4), reused 2 (delta 0), pack-reused 21
Unpacking objects: 100% (34/34), done.
Archive:  ssyp-2022-materials/data/pokemon.zip
  inflating: pokemon/official-jpg-named/abomasnow.jpg  
  inflating: pokemon/official-jpg-named/abra.jpg  
  inflating: pokemon/official-jpg-named/absol.jpg  
  inflating: pokemon/official-jpg-named/accelgor.jpg  
  inflating: pokemon/official-jpg-named/aegislash-shield.jpg  
  inflating: pokemon/official-jpg-named/aerodactyl.jpg  
  inflating: pokemon/official-jpg-named/aggron.jpg  
  inflating: pokemon/official-jpg-named/aipom.jpg  
  inflating: pokemon/official-jpg-named/alakazam.jpg  
  inflating: pokemon/official-jpg-named/alcremie.jpg  
  inflating: pokemon/official-jpg-named/alomomola.jpg  
  inflating: pokemon/official-jpg-named/altaria

In [ ]:
!mkdir checkpoints
!cp /content/drive/MyDrive/awesomemodel__dalle_last.pt /content/checkpoints/awesomemodel__dalle_last.pt  

cp: cannot stat '/content/drive/MyDrive/awesomemodel__dalle_last.pt': No such file or directory


In [ ]:
# Доступные ресурсы
import multiprocessing
import torch
from psutil import virtual_memory

ram_gb = round(virtual_memory().total / 1024**3, 1)

print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device.type)

!nvidia-smi

CPU: 2
RAM GB: 12.7
PyTorch version: 1.10.0+cu111
CUDA version: 11.1
cuDNN version: 8005
device: cuda
Tue Dec 14 18:41:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    46W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------

# Устанавливаем необходимые модели

In [ ]:
!pip install rudalle==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 5.4 MB/s 
     |████████████████████████████████| 60.3 MB 1.2 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 66 kB 4.7 MB/s 
     |████████████████████████████████| 51 kB 276 kB/s 
     |████████████████████████████████| 1.7 MB 65.6 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 2.8 MB 67.4 MB/s 
     |████████████████████████████████| 4.4 MB 58.9 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 244 kB 54.5 MB/s 
     |████████████████████████████████| 585 kB 74.5 MB/s 
     |████████████████████████████████| 117 kB 27.9 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 140 kB 26.9 MB/s 
     |████████████████████████████████| 419

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 29.9 MB/s 
     |████████████████████████████████| 181 kB 67.6 MB/s 
     |████████████████████████████████| 153 kB 73.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d3998612095df75c0ea3333c7654d6f633539ec78766df70b7d779734563292a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


# data to dataloader

In [ ]:
# Импортируем модель
from rudalle.pipelines import generate_images, show, super_resolution, cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae, get_realesrgan, get_ruclip
from rudalle.utils import seed_everything




device = 'cuda'
model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device=device)

vae = get_vae().to(device)
tokenizer = get_tokenizer()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.


Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

vae --> ready


Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer --> ready


In [ ]:

import io
import os
import PIL
import random
import numpy as np
import torch
import torchvision
import transformers
import more_itertools
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF


class RuDalleDataset(Dataset):
    clip_filter_thr = 0.24
    def __init__(
            self,
            file_path,
            csv_path,
            tokenizer,
            resize_ratio=0.75,
            shuffle=True,
            load_first=None,
            caption_score_thr=0.6
    ):
        """ tokenizer - объект с методами tokenizer_wrapper.BaseTokenizerWrapper """
       
        self.text_seq_length = model.get_param('text_seq_length')
        self.tokenizer = tokenizer
        self.target_image_size = 256
        self.image_size=256
        self.samples = []


        self.image_transform = T.Compose([
                T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
                T.RandomResizedCrop(self.image_size,
                                    scale=(1., 1.), 
                                    ratio=(1., 1.)),
                T.ToTensor()
            ])
        
        df = pd.read_csv(csv_path, encoding="ISO-8859-1")
        for caption, f_path  in zip(df['caption'], df['name']):
            try:
              if len(caption)>10 and len(caption)<100 and os.path.isfile(f'{file_path}/{f_path}'):

                self.samples.append([file_path, f_path, caption])
            except:
              pass
        if shuffle:
            np.random.shuffle(self.samples)
            print('Shuffled')
    
    def __len__(self):
        return len(self.samples)

    def load_image(self, file_path, img_name):
        image = PIL.Image.open(f'{file_path}/{img_name}')
        return image

    def __getitem__(self, item):
        item = item % len(self.samples) 
        file_path, img_name, text = self.samples[item]
        try:
          image = self.load_image(file_path, img_name)
          image = self.image_transform(image).to(device)
        except Exception as err: 
            print(err)
            random_item = random.randint(0, len(self.samples) - 1)
            return self.__getitem__(random_item)
        text =  tokenizer.encode_text(text, text_seq_length=self.text_seq_length).squeeze(0).to(device)
        return text, image

In [ ]:
class Args():
    def __init__(self):
     
        self.text_seq_length = model.get_param('text_seq_length')
        self.total_seq_length = model.get_param('total_seq_length')
        self.epochs = 12
        self.save_path='checkpoints/'
        self.model_name = 'awesomemodel_'
        self.save_every = 2000
        self.prefix_length = 10
        self.bs = 1
        self.clip = 0.24
        self.lr = 1e-5
        self.warmup_steps =50
        self.wandb = False
args = Args()
if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

# Модель RuDalle

In [ ]:
from torch.utils.data import Dataset, DataLoader
st = RuDalleDataset(file_path='pokemon/official-jpg-named',
                    csv_path ='gdrive/MyDrive/pokemon_dall_e.csv',tokenizer=tokenizer)
len(st)

Shuffled


786

In [ ]:
train_dataloader = DataLoader(st, batch_size=args.bs, shuffle=True, drop_last=True)

In [ ]:
args.wandb = False

In [ ]:
from transformers import  AdamW, get_linear_schedule_with_warmup
model.train()
optimizer = AdamW(model.parameters(), lr = args.lr)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, 
                                                final_div_factor=500,  
                                                steps_per_epoch=len(train_dataloader), epochs=args.epochs,
                                                pct_start=0.1)




In [ ]:
def freeze(
    model,
    freeze_emb=True,
    freeze_ln=False,
    freeze_attn=False,
    freeze_ff=True,
    freeze_other=True,
):
    for name, p in model.module.named_parameters():
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
    return model

In [ ]:
# Обучение
def train(model,args: Args, train_dataloader: RuDalleDataset):
  
  
  """
  args - аргументы для обучения
  train_dataloader - класс RuDalleDataset - при обращении как к итератору
  возвращает пару текст-картинка (х количество батчей)
  """
  loss_logs = []
  try:
    progress = tqdm(total=len(train_dataloader) * args.epochs, desc='finetuning goes brrr')
    save_counter = 0
    for epoch in range(args.epochs):
      
      for text, images in train_dataloader:
        device = model.get_param('device')
        save_counter+=1
        model.zero_grad()
        attention_mask = torch.tril(torch.ones((args.bs, 1, args.total_seq_length, args.total_seq_length), device=device))
        image_input_ids = vae.get_codebook_indices(images)
        
        input_ids = torch.cat((text, image_input_ids), dim=1) 
        loss, loss_values = model.forward(input_ids, attention_mask, return_loss=True)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        # здесь сохраняются чекпоинты
        if save_counter % args.save_every == 0:
          print(f'Saveing checkpoint here {args.model_name}_dalle_{save_counter}.pt')
          
          plt.plot(loss_logs)
          plt.show()
          torch.save(
                    model.state_dict(),
                    os.path.join(args.save_path,f"{args.model_name}_dalle_{save_counter}.pt")
                    )
        loss_logs+=[loss.item()]
        progress.update()
        progress.set_postfix({"loss": loss.item()})
    
    print(f'Complitly tuned and saved here  {args.model_name}__dalle_last.pt')
    
    plt.plot(loss_logs)
    plt.show()
    
    torch.save(
                model.state_dict(),
                os.path.join(args.save_path,f"{args.model_name}_dalle_last.pt")
                )
  
  except KeyboardInterrupt:
    
    
    print(f'What for did you stopped? Please change model_path to /{args.save_path}/{args.model_name}_dalle_Failed_train.pt')
    plt.plot(loss_logs)
    plt.show()
    
    torch.save(
                model.state_dict(),
                os.path.join(args.save_path,f"{args.model_name}_dalle_Failed_train.pt")
                )
  except Exception as err:
    print(f'Failed with {err}')

In [ ]:
# Можно размораживать разлиичные веса моделей
model = freeze(model = model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False)

train(model, args, train_dataloader)

finetuning goes brrr:   0%|          | 0/9432 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/rudalle/dalle/model.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dtype=torch.long, device=self.device) // self.image_tokens_per_dim
finetuning goes brrr:  20%|█▉        | 1886/9432 [24:24<1:36:59,  1.30it/s, loss=3.37]

In [ ]:
!cp /content/checkpoints/awesomemodel__dalle_last.pt /content/drive/MyDrive/awesomemodel__dalle_last.pt 

# Начинаем генерировать

In [ ]:
!mkdir /content/checkpoints
!cp /content/drive/MyDrive/awesomemodel__dalle_last.pt  /content/checkpoints/awesomemodel__dalle_last.pt 

In [ ]:
# Если хотите загрузить уже обученную (в этом же ноутбуке модель) - раскоментируйте строчку
# model_path = os.path.join('/content/'+args.save_path,f"{args.model_name}_dalle_last.pt")

from rudalle.pipelines import generate_images, show, super_resolution, cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae, get_realesrgan, get_ruclip
from rudalle.utils import seed_everything
import os
import multiprocessing
import torch
from psutil import virtual_memory


device = 'cuda'
model_path = os.path.join('/content/checkpoints/awesomemodel__dalle_last.pt')
model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device=device)
model.load_state_dict(torch.load(model_path)) 
print(f'Loaded from {model_path}')
vae = get_vae().to(device)
tokenizer = get_tokenizer()

Downloading:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
Loaded from /content/checkpoints/awesomemodel__dalle_last.pt
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.


Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

vae --> ready


Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer --> ready


In [ ]:
pil_images = []
scores = []
images_num = 3
text = 'Рандомный покемон'

img_count = 0
for top_k, top_p in [
        (2048, 0.995),
        (1536, 0.99),
        (1024, 0.99),
        (1024, 0.98),
        (512, 0.97),
        (384, 0.96),
        (256, 0.95),
        (128, 0.95), 
    ]:
      _pil_images, _scores = generate_images(text, tokenizer, model, vae, top_k=top_k, images_num=images_num, top_p=top_p)
      for images in _pil_images:
            images.save(f"./gen_images/{img_count:03d}.png")
            img_count += 1
      show(_pil_images, 3, size=28)
      pil_images += _pil_images
show([pil_image for pil_image in pil_images], 3, size=56)

In [ ]:
import shutil
from IPython.display import clear_output

!mkdir /content/batch
batch_start = 816

mkdir: cannot create directory ‘/content/batch’: File exists


In [ ]:
text = 'Покемон'
images_num = 8

while batch_start < 5000:
  for top_k, top_p in [
          (2048, 0.995),
          (1536, 0.99),
          (1024, 0.99),
          (1024, 0.98),
          (512, 0.97),
          (384, 0.96),
          (256, 0.95),
          (128, 0.95), 
      ]:
        _pil_images, _scores = generate_images(text, tokenizer, model, vae, top_k=top_k, images_num=images_num, top_p=top_p)
        for images in _pil_images:
              images.save(f"./batch/{batch_start:04d}.png")
              shutil.copy(f"/content/batch/{batch_start:04d}.png", f"/content/drive/MyDrive/ai_pokemon_batch/{batch_start:04d}.png" )
              batch_start += 1
        clear_output(wait=True)


  0%|          | 0/1024 [00:00<?, ?it/s]